In [2]:
%pip install --upgrade chardet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.4/199.4 kB 20.5 kB/s eta 0:00:0000:0100:02


In [7]:
from transformers import pipeline
from transformers import AutoTokenizer,GenerationConfig
from transformers import AutoModelForSeq2SeqLM,AutoModelForCausalLM

# import huggingface_hub
import csv
# import time
# from multiprocessing.pool import ThreadPool
# import requests
# import os
# import PIL.Image as Image
# from io import BytesIO 
# import numpy as np
from tqdm import tqdm
# import shutil

import ast


ImportError: cannot import name 'get_full_repo_name' from 'huggingface_hub' (/home/mty/miniconda3/envs/py39/lib/python3.9/site-packages/huggingface_hub/__init__.py)

In [10]:
CUDA_VISIBLE_DEVICES = 5
device = "cuda:0"
base_file = "./data/eval_images_coarse-grained.csv"

In [11]:
tokenizer = AutoTokenizer.from_pretrained("Gustavosta/MagicPrompt-Stable-Diffusion")
model = AutoModelForCausalLM.from_pretrained("Gustavosta/MagicPrompt-Stable-Diffusion")
gpt2_pipe = pipeline("text-generation", model= model, tokenizer = tokenizer, device = device)

In [ ]:

output = []
with open(base_file, 'r+') as f:
    data = csv.DictReader(f)
    data = [r for r in data]
    pbar = tqdm(total=len(data))
    for row in data:
        prompt_list = ast.literal_eval(row["coarse-grained prompt"])
        prompt = prompt_list[1]["prompt"]
        gpt2_ans = gpt2_pipe(prompt,do_sample=True,num_return_sequences=5,num_beams=10)
        for i,_ in enumerate(gpt2_ans):
            row[f"ans_{i}"] = _
        output.append(row)
        pbar.update(1)
    pbar.close()

  0%|          | 10/8192 [00:06<1:27:28,  1.56it/s]/usr/local/lib/python3.9/dist-packages/transformers/pipelines/base.py:1081: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
  0%|          | 40/8192 [00:25<1:28:19,  1.54it/s]Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [ ]:
with open(f"./data/train_images_coarse-grained.csv.csv", 'w', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=output[0].keys())
    writer.writeheader()
    writer.writerows(output)